<a href="https://colab.research.google.com/github/pranan71/AIMLtraining_Prakash_Narayanan_M/blob/main/audioSpectrogram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import wave
import pylab
from pathlib import Path
from scipy import signal
from scipy.io import wavfile
from sklearn.metrics import confusion_matrix
import itertools

FileNotFoundError: ignored

In [ ]:

# Set paths to input and output data
INPUT_DIR = '/kaggle/input/free-spoken-digits/free-spoken-digit-dataset-master/recordings/'
OUTPUT_DIR = '/kaggle/working/'

In [ ]:
# Print names of 10 WAV files from the input path
parent_list = os.listdir(INPUT_DIR)
for i in range(10):
    print(parent_list[i])